<a href="https://colab.research.google.com/github/pgordin/OptDisc2024/blob/main/graph_class_SN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from random import random

In [2]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_array(cls, graph: np.array, nodes: list = None):
        if nodes is None:
            nodes = [*range(1, len(graph) + 1)]
        return cls.from_dict(
            cls._array_to_dict(graph, nodes)
        )

    @staticmethod
    def from_file(filename: str, directed = 0):
        """
        Generates a graphs from a text file, where each line defines one edge.\n
        Filename is a file path.
        """
        graph = Graph()
        file = open(filename, "r")
        for line in file:
            words = line.split() # nie potrzeba .strip()
            if len(words) == 1:
                graph.add_node(words[0])
            elif len(words) > 1: # more than two words -- we use first two
                if directed:
                    graph.add_arc([words[0], words[1]])
                else:
                    graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(nodes_num: int, prob: float):
        """
        Generates a random graph provided a number of nodes and probability of generating an edge.
        """
        rand_graph = Graph()
        for i in range(1, nodes_num + 1):
            for j in range(1, i):
                if random() < prob:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(nodes_num: int):
        """
        Generates a cycle provided a number of nodes.
        """
        cycle = Graph()
        nodes = [*range(1, nodes_num + 1)]
        for i in nodes:
            cycle.add_edge([nodes[i-2], nodes[i-1]])
        cycle.graph = dict(sorted(cycle.graph.items()))
        return cycle

    def to_file(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")
        for v in self.graph:
            neigh_list = f"{v}:"
            for u in self.graph[v]:
                neigh_list += f" {u}"
            neigh_list += "\n"
            file.write(neigh_list)
        file.close()

    def nodes(self) -> list:
        """
        Returns list of nodes of a graph.
        """
        return [*self.graph.keys()]

    def array(self) -> np.array:
        """
        Returns the graph in array form.
        """
        return self._dict_to_array(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v} -> {self.graph[v]}\n"
        return res

    def add_node(self, node):
        """
        Adds a node to a graph.
        """
        if node not in self.graph:
            self.graph[node] = []

    def del_node(self, node):
        """
        Recursively removes a node from a graph.
        """
        if node in self.graph:
            self.graph.pop(node)
            for key in [*self.graph.keys()]:
                if node in self.graph[key]:
                    self.graph[key].remove(node)

    def add_arc(self, arc: list):
        """
        Adds arc to a graph provided a list of nodes.
        """
        u, v = arc
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Adds edge to a graph provided a list of nodes.
        """
        u, v = edge
        if u == v:
            raise ValueError("Pętla!")
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    def _array_to_dict(arr: np.array, nodes: list) -> dict:
        """
        Converts a graph in array form to a graph in dict form.
        """
        res_dict = {}
        for i, node in enumerate(nodes):
            neighbours = [nodes[j] for j, edge in enumerate(arr[i]) if edge]
            res_dict[node] = neighbours
        return res_dict

    def _dict_to_array(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to a graph in array form.
        """
        n = len(_dict)
        nodes = [*_dict.keys()]
        res_arr = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (nodes.index(u), nodes.index(v))
            for u, row in _dict.items() for v in row
        ]:
            res_arr[u][v] += 1
        return res_arr


# Przykłady wykorzystania

In [3]:
graph = Graph.from_dict(
    {
        'a': ['b', 'c'],
        'b': ['a', 'd', 'e'],
        'c': ['a', 'f'],
        'd': ['b', 'f'],
        'e': ['b'],
        'f': ['c', 'd'],
        'g': [],
    }
)

graph_array = graph.array()
graph_nodes = graph.nodes()

print(graph)
print(graph_array)
print(graph_nodes)

a -> ['b', 'c']
b -> ['a', 'd', 'e']
c -> ['a', 'f']
d -> ['b', 'f']
e -> ['b']
f -> ['c', 'd']
g -> []

[[0 1 1 0 0 0 0]
 [1 0 0 1 1 0 0]
 [1 0 0 0 0 1 0]
 [0 1 0 0 0 1 0]
 [0 1 0 0 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0]]
['a', 'b', 'c', 'd', 'e', 'f', 'g']


In [4]:
random_graph = Graph.random_graph(10, 1/3)
print(random_graph)

2 -> [1, 3, 7, 9]
1 -> [2, 8]
3 -> [2, 4, 5, 7]
4 -> [3, 5, 9, 10]
5 -> [3, 4, 8]
7 -> [2, 3]
8 -> [1, 5]
9 -> [2, 4, 6]
6 -> [9]
10 -> [4]



In [ ]:
cycle = Graph.cycle(10)
print(cycle)

1 -> [10, 2]
2 -> [1, 3]
3 -> [2, 4]
4 -> [3, 5]
5 -> [4, 6]
6 -> [5, 7]
7 -> [6, 8]
8 -> [7, 9]
9 -> [8, 10]
10 -> [1, 9]



In [5]:
%%writefile lista.txt
A B
B C
B D
D C
E
F

Writing lista.txt


In [6]:
file_graph = Graph.from_file("./lista.txt")
print(file_graph)

A -> ['B']
B -> ['A', 'C', 'D']
C -> ['B', 'D']
D -> ['B', 'C']
E -> []
F -> []



In [7]:
file_graph.add_edge(["E", "F"])
file_graph.to_file("lista1.txt")

In [8]:
%cat lista1.txt

A: B
B: A C D
C: B D
D: B C
E: F
F: E
